In [0]:
import tensorflow as tf
import numpy as np

data = np.loadtxt('./data.csv', delimiter=',', unpack=True, dtype = 'float32')
x_data = np.transpose(data[0:4])
y_data = np.transpose(data[4:])


numpy library의 loadtext함수를 이용하여 데이터를 불러온다   
transpose,unpack 둘다 전치행렬을 만듬( M x N -> N x M)


In [0]:
global_step = tf.Variable(0, trainable = False, name='global_step')

학습횟수를 카운트하는 변수 추가   
학스에 사용되지 않으므로 trainable = False이다.

In [0]:
X= tf.placeholder(tf.float32)
Y=tf.placeholder(tf.float32)

W1 = tf.Variable(tf.random_uniform([4,10],-1.,1.))
L1 = tf.nn.relu(tf.matmul(X,W1))

W2 = tf.Variable(tf.random_uniform([10,20],-1.,1.))
L2 = tf.nn.relu(tf.matmul(L1,W2))

W3 = tf.Variable(tf.random_uniform([20,5],-1.,1.))
model = tf.matmul(L2,W3)

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=model, labels=Y))

optimizer = tf.train.AdamOptimizer(learning_rate = 0.01)
train_op = optimizer.minimize(cost, global_step = global_step)

```train_op = optimizer.minimize(cost, global_step = global_step)```   
마지막으로 학습시킬떄 앞에서 만든 global_step 에 global_step을 넣어줌으로 학습이 진행될때마다 1씩 증가

In [0]:
sess = tf.Session()
saver = tf.train.Saver(tf.global_variables())

ckpt = tf.train.get_checkpoint_state('./models')
if ckpt and tf.train.checkpoint_exists(ckpt.model_checkpoint_path):
  saver.restore(sess, ckpt.model_checkpoint_path)
else:
  sess.run(tf.global_variables_initializer())

tf.global_variables는 앞서 정의한 변수들을 가져오는 함수 입니다.   
이 함수를 써 앞서 정의한 변수들을 모두 가져와서 이후 이 변수들을 파일에 저장하거나 이전에 학습한 결과를 불러와 담는 변수들로 사용합니다.   

다음으로 checkpoint를 설정합니다.   
./model폴더에 만약 기존에 학습해둔 모델이 있을시 ```saver.restore```함수
를 통해 학습된 값을 불러오고, 없을 시 변수들을 새로 초기화 합니다.

In [17]:
for step in range(10):
  sess.run(train_op, feed_dict={X: x_data, Y: y_data})

  print('Step: %d,'%sess.run(global_step), 'Cost: %.3f' %sess.run(cost,feed_dict={X: x_data, Y: y_data}))

Step: 41, Cost: 0.064
Step: 42, Cost: 0.059
Step: 43, Cost: 0.054
Step: 44, Cost: 0.050
Step: 45, Cost: 0.046
Step: 46, Cost: 0.042
Step: 47, Cost: 0.039
Step: 48, Cost: 0.036
Step: 49, Cost: 0.034
Step: 50, Cost: 0.031


global_step을 print시 global_step이 tensor 타입이므로 

In [18]:
saver.save(sess, './model/dnn.ckpt', global_step=global_step)

'./model/dnn.ckpt-50'

In [19]:
prediction = tf.argmax(model,1)
target = tf.argmax(Y,1)
print('예측값:', sess.run(prediction, feed_dict={X: x_data}))
print('결과값:', sess.run(target, feed_dict={Y: y_data}))
is_correct =tf.equal(prediction, target)
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))
print('정확도:%.2f'% sess.run(accuracy*100, feed_dict={X:x_data,Y:y_data}))

예측값: [0 2 0 1 1 0 0 0 1 0 1 0 1 2 1 0 2 0 0 1 0 0 0 1 0 0 0 0 2 0 1 1 0 1 0 1 0
 0 3 0 0 0 1 0 0 1 0 0 1 2 1 1 0 0 0 2 1 0 1 0 2 2 1 2 1 1 1 2 1 0 1 1 1 1
 0 1 2 1 2 0 1 1 0 0 0 1 2 0 2 0 0 1 0 0 0 2 0 4 1 2]
결과값: [0 2 0 1 1 0 0 0 1 0 1 0 1 2 1 0 2 0 0 1 0 0 0 1 0 0 0 0 2 0 1 1 0 1 0 1 0
 0 3 0 0 0 1 0 0 1 0 0 1 2 1 1 0 0 0 2 1 0 1 0 2 2 1 2 1 1 1 2 1 0 1 1 1 1
 0 1 2 1 2 0 1 1 0 0 0 1 2 0 2 0 0 1 0 0 0 2 0 4 1 2]
정확도:100.00
